In [1]:
import os
import json
import numpy as np
import pandas as pd
import umap
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score, silhouette_score, adjusted_rand_score, \
    normalized_mutual_info_score
from collections import Counter

In [2]:
def read_csv_files_to_dict(base_path, dimension, file_prefix, missing_percentages, strategies=None):
    dataframes_dict = {}

    for percentage in missing_percentages:
        percentage_key = f"{int(percentage * 100)}%"

        if dimension == 'Completeness':
            dataframes_dict[percentage_key] = {}

            if strategies is not None:
                for strategy_name, strategy_func, subfolder in strategies:
                    file_path = f"{base_path}/{dimension}/{subfolder}/{file_prefix}_{percentage_key}.csv"

                    try:
                        df = pd.read_csv(file_path)
                        dataframes_dict[percentage_key][strategy_name] = df
                    except FileNotFoundError:
                        print(f"File not found: {file_path}")
                        dataframes_dict[percentage_key][strategy_name] = None
            else:
                print("No strategies provided for Completeness dimension.")
        elif dimension == 'Unicity':
            file_path = f"{base_path}/{dimension}/{file_prefix}_{percentage_key}_2x.csv"
            try:
                df = pd.read_csv(file_path)
                dataframes_dict[percentage_key] = df
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                dataframes_dict[percentage_key] = None
        else:
            file_path = f"{base_path}/{dimension}/{file_prefix}_{percentage_key}.csv"
            try:
                df = pd.read_csv(file_path)
                dataframes_dict[percentage_key] = df
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                dataframes_dict[percentage_key] = None

    return dataframes_dict

def convert_to_float(obj):
    """
    Convertit les types de données non JSON-serializables en types natifs Python.
    """
    if isinstance(obj, np.float32) or isinstance(obj, np.float64):
        return float(obj)
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

def update_json_results(output_path, model_name, metrics, pollution_percentage_levels):
    # Charger le fichier JSON existant, ou initialiser une nouvelle structure si le fichier n'existe pas
    if os.path.exists(output_path):
        with open(output_path, 'r') as json_file:
            results_dict = json.load(json_file)
    else:
        results_dict = {
            "models": []
        }

    # Trouver ou ajouter l'entrée pour le modèle spécifié
    model_entry = next((model for model in results_dict["models"] if model["model"] == model_name), None)

    if not model_entry:
        model_entry = {
            "model": model_name,
            "pollution_metrics": []
        }
        results_dict["models"].append(model_entry)

    # Mise à jour pour la pollution à 0 (df_clean)
    pollution_percentage = 0
    existing_entry = next((item for item in model_entry["pollution_metrics"] if item["pollution_percentage"] == pollution_percentage), None)

    if existing_entry:
        # Remplacer les métriques
        existing_entry["metrics"] = {
            # "silhouette score": convert_to_float(metrics["Silhouette_Score_Clean"]),
            "stability indexes": 1.0,
            "ARI score": 1.0
        }
    else:
        # Ajouter une nouvelle entrée pour ce pourcentage
        model_entry["pollution_metrics"].append({
            "pollution_percentage": pollution_percentage,
            "metrics": {
                # "silhouette score": convert_to_float(metrics["Silhouette_Score_Clean"]),
                "stability indexes": 1.0,
                "ARI score": 1.0
            }
        })

    # Mise à jour pour les autres niveaux de pollution
    for i, pollution_percentage in enumerate(pollution_percentage_levels):
        pollution_percentage = int(pollution_percentage*100)
        existing_entry = next((item for item in model_entry["pollution_metrics"] if item["pollution_percentage"] == pollution_percentage), None)

        if existing_entry:
            # Remplacer les métriques
            existing_entry["metrics"] = {
                # "silhouette score": convert_to_float(metrics["Silhouette_Score_Noisy"][i]),
                "stability indexes": convert_to_float(metrics["Stability_Index"][i]),
                "ARI score": convert_to_float(metrics["ARI"][i])
            }
        else:
            # Ajouter une nouvelle entrée pour ce pourcentage
            model_entry["pollution_metrics"].append({
                "pollution_percentage": pollution_percentage,
                "metrics": {
                    # "silhouette score": convert_to_float(metrics["Silhouette_Score_Noisy"][i]),
                    "stability indexes": convert_to_float(metrics["Stability_Index"][i]),
                    "ARI score": convert_to_float(metrics["ARI"][i])
                }
            })

    # Écrire les résultats mis à jour dans le fichier JSON
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")


def prepare_retail_data(df):
    # Séparation des caractéristiques numériques et catégorielles
    numeric_features = df.select_dtypes(include='number')
    categorical_features = df.select_dtypes(include='object')

    # Transformation des données numériques
    scaler = StandardScaler()
    numeric_data_scaled = scaler.fit_transform(numeric_features)

    # Transformation des données catégorielles en utilisant pd.get_dummies
    categorical_data_encoded = pd.get_dummies(categorical_features, drop_first=True)

    # Combinaison des données numériques et catégorielles transformées
    df_preprocessed = pd.concat([pd.DataFrame(numeric_data_scaled, columns=numeric_features.columns).reset_index(drop=True),
                                 categorical_data_encoded.reset_index(drop=True)], axis=1)

    return df_preprocessed

def compute_metrics(df_clean, df_noisy_dict, clustering_algo=DBSCAN(eps=0.5, min_samples=5)):
    metrics_results = {
        "Stability_Index": [],
        "ARI": [],
    }

    # 1. Réduction de dimensionnalité sur les données propres avec un nouveau modèle UMAP
    print("Réduction de dimensionnalité sur les données propres")
    umap_model_clean = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2)
    embedding_clean = umap_model_clean.fit_transform(df_clean)

    # 2. Clustering sur les données propres avec DBSCAN
    print("Clustering sur les données propres")
    cluster_labels_clean = clustering_algo.fit_predict(embedding_clean)

    for key in df_noisy_dict.keys():
        df_noisy = df_noisy_dict[key]

        if df_noisy is not None:
            # 3. Réduction de dimensionnalité sur les données bruitées avec un nouveau modèle UMAP
            print(f"Réduction de dimensionnalité sur les données bruitées: {key}")
            umap_model_noisy = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2)
            embedding_noisy = umap_model_noisy.fit_transform(df_noisy)

            # 4. Clustering sur les données bruitées avec DBSCAN
            print(f"Clustering sur les données bruitées: {key}")
            cluster_labels_noisy = clustering_algo.fit_predict(embedding_noisy)

            # 5. Calcul des métriques

            # Adjusted Rand Index (ARI)
            print("Calcul du Adjusted Rand Index (ARI) sur les données bruitées")
            ari = adjusted_rand_score(cluster_labels_clean, cluster_labels_noisy)
            metrics_results["ARI"].append(ari)

            # Stability Index (SI) - en utilisant l'AMI comme proxy
            print("Calcul du Stability Index (SI) sur les données bruitées")
            stability_index = normalized_mutual_info_score(cluster_labels_clean, cluster_labels_noisy)
            metrics_results["Stability_Index"].append(stability_index)
        else:
            # Si le dataframe n'est pas disponible, on ajoute None pour ce niveau de bruit
            metrics_results["ARI"].append(None)
            metrics_results["Stability_Index"].append(None)

    return metrics_results

## Feature Accuracy

In [3]:
output_path = "../../Results/Retail/Feature Accuracy.json"
model_name = "DBSCAN"

# Chargement du DataSet clean
retail_df_clean = pd.read_csv('../../Data/Clustering/retail Data/retail_data_clean.csv')
retail_df_clean = prepare_retail_data(retail_df_clean)

# Chargement des DataSet pollués
pollution_percentage_levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
retail_df_noisy_dict = read_csv_files_to_dict('../../Data/Clustering/retail Data', 'Feature Accuracy', 'retail', pollution_percentage_levels)

for key in retail_df_noisy_dict.keys():
    if retail_df_noisy_dict[key] is not None:
        retail_df_noisy_dict[key] = prepare_retail_data(retail_df_noisy_dict[key])

metrics = compute_metrics(retail_df_clean, retail_df_noisy_dict)

update_json_results(output_path, model_name, metrics, pollution_percentage_levels)

Réduction de dimensionnalité sur les données propres


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Clustering sur les données propres
Réduction de dimensionnalité sur les données bruitées: 10%
Clustering sur les données bruitées: 10%


KeyboardInterrupt: 